In [1]:
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np

base_dir = "Downloads/cat and dog dataset"
train_path_cats = base_dir + "/training_set/training_set/cats"
train_path_dogs = base_dir + "/training_set/training_set/dogs"

test_path_cats = base_dir + "/test_set/test_set/cats"        
test_path_dogs = base_dir + "/test_set/test_set/dogs"

# print(train_path_cats)
# print(train_path_dogs)
# print(test_path_cats)
# print(test_path_dogs)

In [2]:
import os
import matplotlib.pyplot as plt
img_size = (128, 128)
def load_data_from_folder(folder_path):
    images=[]
    for filename in os.listdir(folder_path):
        if filename.endswith(('jpg', 'png')):
            img_path = os.path.join(folder_path, filename)
    #         print(img_path)
            img = Image.open(img_path)
            img = img.resize(img_size, Image.Resampling.LANCZOS)
            img = img.convert('L')
            
            img = np.array(img)
            img = img.reshape(1, 128, 128)
    #         plt.imshow(img, cmap='gray')
    #         plt.axis('off')
    #         plt.show()
            images.append(img)
    
    return images
        
train_cat = load_data_from_folder(train_path_cats)
t_cat_label = np.zeros(len(train_cat))
train_dog = load_data_from_folder(train_path_dogs)
t_dog_label = np.ones(len(train_dog))

train_data = np.vstack((train_cat, train_dog))
train_label = np.hstack((t_dog_label, t_cat_label))
# print(np.shape(train_dog))
# print(np.shape(train_cat))
# print(np.shape(train_data))
print(np.shape(train_data))
print(np.shape(train_label))

index = np.arange(8005)
print(index)
np.random.shuffle(index)
print(index)
train_data = train_data[index]
train_label = train_label[index]

(8005, 1, 128, 128)
(8005,)
[   0    1    2 ... 8002 8003 8004]
[2037  149 6138 ... 7050 7585 4045]


In [3]:
test_cat = load_data_from_folder(test_path_cats)
t_cat_label = np.zeros(len(test_cat))
test_dog = load_data_from_folder(test_path_dogs)
t_dog_label = np.ones(len(test_dog))

test_data = np.vstack((test_cat, test_dog))
test_label = np.hstack((t_dog_label, t_cat_label))
train_data = train_data/255
test_data = test_data/255
# print(np.shape(train_dog))
# print(np.shape(train_cat))
# print(np.shape(train_data))
print(np.shape(test_data))
print(np.shape(test_label))

index = np.arange(len(test_data))
np.random.shuffle(index)
test_data = test_data[index]
test_label = test_label[index]

(2023, 1, 128, 128)
(2023,)


In [4]:
import torch

train_data = torch.tensor(train_data, dtype=torch.float32)
train_label = torch.tensor(train_label, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.float32)
test_label = torch.tensor(test_label, dtype=torch.long)
train_data = train_data.view(8005, 1, 128, 128)
train_label = train_label.view(8005)
test_data = test_data.view(2023, 1, 128, 128)
test_label = test_label.view(2023)
print(train_data.shape)
print(train_label.shape)

torch.Size([8005, 1, 128, 128])
torch.Size([8005])


In [5]:
from torch.utils.data import TensorDataset, DataLoader

train = TensorDataset(train_data, train_label)
print(train)
trainset = DataLoader(train, batch_size = 32, shuffle=True)
# for data in trainset:
#     x, y = data
#     print(x.shape)

In [6]:
for data in trainset:
    print(data)
    break

[tensor([[[[0.2667, 0.2627, 0.2745,  ..., 0.2941, 0.2784, 0.2745],
          [0.2667, 0.2667, 0.2706,  ..., 0.2941, 0.2824, 0.2784],
          [0.2667, 0.2627, 0.2667,  ..., 0.2863, 0.2863, 0.2824],
          ...,
          [0.3490, 0.3490, 0.3451,  ..., 0.3294, 0.3294, 0.3216],
          [0.3412, 0.3412, 0.3412,  ..., 0.3176, 0.3216, 0.3137],
          [0.3373, 0.3373, 0.3412,  ..., 0.3137, 0.3098, 0.3020]]],


        [[[0.6784, 0.6745, 0.5255,  ..., 0.1373, 0.1529, 0.1647],
          [0.4471, 0.7098, 0.7020,  ..., 0.1451, 0.1569, 0.1725],
          [0.2314, 0.5725, 0.6627,  ..., 0.1490, 0.1647, 0.1647],
          ...,
          [0.3412, 0.3412, 0.3373,  ..., 0.3961, 0.3059, 0.2980],
          [0.3176, 0.3412, 0.3333,  ..., 0.7725, 0.4667, 0.3843],
          [0.3176, 0.3373, 0.3255,  ..., 0.6706, 0.6235, 0.5882]]],


        [[[0.7686, 0.7608, 0.7529,  ..., 0.2118, 0.2078, 0.2000],
          [0.7529, 0.7451, 0.7451,  ..., 0.1961, 0.1961, 0.2039],
          [0.7529, 0.7412, 0.7490,  .

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.l2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.l3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.l5 = nn.Linear(128*16*16, 512)
        self.l6 = nn.Linear(512, 2)
#         self.l5 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.l1(x)))
        x = self.pool(F.relu(self.l2(x)))
        x = self.pool(F.relu(self.l3(x)))
#         x = self.pool(F.relu(self.l4(x)))
        x = x.view(-1, 128*16*16)
        x = F.relu(self.l5(x))
#         x = self.dropout(x)
#         x = F.relu(self.l4(x))
        x = F.log_softmax(self.l6(x), dim=1)
        return x
    
model = cnn()

In [11]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=0.001)
criterian = nn.CrossEntropyLoss()

In [13]:
EPOCH = 20
for epochs in range(EPOCH):
    for data in trainset:
        
        x,y = data
        optimizer.zero_grad()
        
        new_o = model(x)
        loss = criterian(new_o, y)
        loss.backward()
        
        optimizer.step()
    print(f'Epochs: {epochs}, loss: {loss.item()}')

Epochs: 0, loss: 0.5632365345954895
Epochs: 1, loss: 0.154405415058136
Epochs: 2, loss: 0.11312683671712875
Epochs: 3, loss: 0.2969934940338135
Epochs: 4, loss: 0.12176699936389923
Epochs: 5, loss: 0.022735456004738808
Epochs: 6, loss: 0.10445155203342438
Epochs: 7, loss: 0.010300679132342339
Epochs: 8, loss: 0.001051824539899826
Epochs: 9, loss: 0.0028567102272063494
Epochs: 10, loss: 0.0006886792834848166
Epochs: 11, loss: 0.11909200996160507
Epochs: 12, loss: 0.003712850157171488
Epochs: 13, loss: 5.5162970966193825e-05
Epochs: 14, loss: 0.006021395791321993
Epochs: 15, loss: 0.004009475000202656
Epochs: 16, loss: 0.10645373165607452
Epochs: 17, loss: 0.0004662802384700626
Epochs: 18, loss: 0.0003107603988610208
Epochs: 19, loss: 0.006553503219038248
Epochs: 20, loss: 7.098956120898947e-05
Epochs: 21, loss: 0.0007867346284911036
Epochs: 22, loss: 2.8847751309513114e-05
Epochs: 23, loss: 5.626635811495362e-06
Epochs: 24, loss: 0.0834660679101944
Epochs: 25, loss: 1.049039155986975e-0

In [14]:
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    return correct

test = TensorDataset(test_data, test_label)
testset = DataLoader(test, batch_size=2023, shuffle=True)
net_correct = 0
net_accuracy = 0

for data in testset:
    x, y = data
    output = model(x)
    net_correct += calculate_accuracy(output, y)
    
net_accuracy = net_correct/2023
print(net_correct)
print(net_accuracy)

1570
0.7760751359367276


In [15]:
net_correct = 0
net_accuracy = 0

for data in trainset:
    x, y = data
    output = model(x)
    net_correct += calculate_accuracy(output, y)
    
net_accuracy = net_correct/8005
print(net_correct)
print(net_accuracy)

7971
3.940187839841819
